In [2]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
data = pd.read_csv("C:/Users/procns/Desktop/인공지능/HAR/PAMAP/pamap_features.csv")

In [6]:
data

,Unnamed: 0,sub_id,activity_id,act_level,hr_mean,hr_mean_normal,hr_std,hr_std_normal,hand_tmp_mean,hand_tmp_std,...,ankle_acc_xz_cor,hand_gyr_xy_cor,hand_gyr_yz_cor,hand_gyr_xz_cor,chest_gyr_xy_cor,chest_gyr_yz_cor,chest_gyr_xz_cor,ankle_gyr_xy_cor,ankle_gyr_yz_cor,ankle_gyr_xz_cor
0,0,1,1,light,103.951172,1.386016,0.805429,0.010739,30.523682,0.030320,...,0.314868,-0.531362,-0.581758,0.530794,0.230082,-0.481656,-0.604763,0.325803,-0.294619,-0.170427
1,1,1,1,light,104.509766,1.393464,0.952667,0.012702,30.535889,0.030904,...,0.486994,-0.282832,-0.248817,0.230088,-0.118856,-0.139842,-0.481102,0.194234,-0.368513,-0.292237
2,2,1,1,light,105.253906,1.403385,1.094676,0.014596,30.548096,0.026321,...,0.588619,-0.162304,0.219523,0.154424,-0.130829,-0.111774,-0.385171,0.214233,-0.311803,-0.179082
3,3,1,1,light,105.839844,1.411198,1.069291,0.014257,30.555542,0.019659,...,0.499454,-0.300674,0.085189,-0.138638,0.257661,-0.379470,-0.646042,0.187620,-0.438468,-0.192798
4,4,1,1,light,106.339844,1.417865,0.853795,0.011384,30.558350,0.015562,...,0.105384,-0.167682,0.148621,-0.260610,0.276035,-0.401572,-0.531046,0.008690,-0.389506,-0.293527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13068,16814,9,24,vigorous,153.171875,2.836516,0.367739,0.006810,25.125000,0.000000,...,-0.701407,-0.735185,0.362872,-0.583763,-0.052483,-0.045138,-0.172941,0.239495,0.826156,0.110417
13069,16815,9,24,vigorous,153.091797,2.835033,0.282523,0.005232,25.125000,0.000000,...,-0.686521,-0.704236,0.341362,-0.601732,-0.045563,-0.079220,-0.078190,0.152746,0.813039,0.030027
13070,16816,9,24,vigorous,153.035156,2.833984,0.174267,0.003227,25.125000,0.000000,...,-0.640646,-0.645367,0.318070,-0.596354,-0.036907,-0.027967,-0.070725,0.087238,0.794210,-0.039155
13071,16817,9,24,vigorous,153.346680,2.839753,0.670464,0.012416,25.125000,0.000000,...,-0.578442,-0.522717,0.368583,-0.434512,-0.094335,0.058254,-0.172321,-0.004136,0.805680,-0.142349


In [7]:
activity_id = {0: 'transient', 1:'lying', 2:'sitting', 3:'standing',
              4:'walking', 5:'running', 6:'cycling', 7:'Nordic walking',
              9:'watching TV', 10:'computer work', 11:'car driving',
              12:'ascending stairs', 13:'descending stairs', 16:'vacuum cleaning',
              17:'ironing', 18:'folding laundry', 19:'house cleaning',
              20:'playing soccer', 24:'rope jumping'}

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13073 entries, 0 to 13072
Data columns (total 68 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        13073 non-null  int64  
 1   sub_id            13073 non-null  int64  
 2   activity_id       13073 non-null  int64  
 3   act_level         13073 non-null  object 
 4   hr_mean           13073 non-null  float64
 5   hr_mean_normal    13073 non-null  float64
 6   hr_std            13073 non-null  float64
 7   hr_std_normal     13073 non-null  float64
 8   hand_tmp_mean     13073 non-null  float64
 9   hand_tmp_std      13073 non-null  float64
 10  hand_acc_x_mean   13073 non-null  float64
 11  hand_acc_x_std    13073 non-null  float64
 12  hand_acc_y_mean   13073 non-null  float64
 13  hand_acc_y_std    13073 non-null  float64
 14  hand_acc_z_mean   13073 non-null  float64
 15  hand_acc_z_std    13073 non-null  float64
 16  hand_gyr_x_mean   13073 non-null  float6

In [13]:
df = data.drop(['Unnamed: 0','sub_id','act_level','hr_mean','hr_mean_normal','hr_std','hr_std_normal'], axis = 1)
X = df.drop(['activity_id'], axis = 1)
y = df['activity_id']

In [14]:
X

,hand_tmp_mean,hand_tmp_std,hand_acc_x_mean,hand_acc_x_std,hand_acc_y_mean,hand_acc_y_std,hand_acc_z_mean,hand_acc_z_std,hand_gyr_x_mean,hand_gyr_x_std,...,ankle_acc_xz_cor,hand_gyr_xy_cor,hand_gyr_yz_cor,hand_gyr_xz_cor,chest_gyr_xy_cor,chest_gyr_yz_cor,chest_gyr_xz_cor,ankle_gyr_xy_cor,ankle_gyr_yz_cor,ankle_gyr_xz_cor
0,30.523682,0.030320,-1.879247,3.534154,4.788385,2.567330,6.990495,1.965119,0.107901,0.466133,...,0.314868,-0.531362,-0.581758,0.530794,0.230082,-0.481656,-0.604763,0.325803,-0.294619,-0.170427
1,30.535889,0.030904,-0.185149,1.377860,5.695728,0.972648,7.785865,0.751240,-0.009273,0.381659,...,0.486994,-0.282832,-0.248817,0.230088,-0.118856,-0.139842,-0.481102,0.194234,-0.368513,-0.292237
2,30.548096,0.026321,0.319354,0.447698,5.626324,0.819042,7.950312,0.582057,-0.037028,0.274945,...,0.588619,-0.162304,0.219523,0.154424,-0.130829,-0.111774,-0.385171,0.214233,-0.311803,-0.179082
3,30.555542,0.019659,0.322273,0.326337,5.483047,0.568834,8.098098,0.372352,0.029006,0.152585,...,0.499454,-0.300674,0.085189,-0.138638,0.257661,-0.379470,-0.646042,0.187620,-0.438468,-0.192798
4,30.558350,0.015562,0.316246,0.102791,5.702229,0.209482,7.966321,0.143267,0.009624,0.092451,...,0.105384,-0.167682,0.148621,-0.260610,0.276035,-0.401572,-0.531046,0.008690,-0.389506,-0.293527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13068,25.125000,0.000000,1.388376,4.245269,9.146590,12.844180,-2.399859,5.126979,0.273583,1.697603,...,-0.701407,-0.735185,0.362872,-0.583763,-0.052483,-0.045138,-0.172941,0.239495,0.826156,0.110417
13069,25.125000,0.000000,1.423980,4.319272,9.157108,13.125012,-2.583755,5.019451,0.290182,1.715815,...,-0.686521,-0.704236,0.341362,-0.601732,-0.045563,-0.079220,-0.078190,0.152746,0.813039,0.030027
13070,25.125000,0.000000,1.214250,4.700313,9.182128,14.451389,-2.597204,5.610332,0.244093,1.770493,...,-0.640646,-0.645367,0.318070,-0.596354,-0.036907,-0.027967,-0.070725,0.087238,0.794210,-0.039155
13071,25.125000,0.000000,-0.232018,5.082784,8.528821,13.341294,-1.575550,5.546635,0.113382,1.799821,...,-0.578442,-0.522717,0.368583,-0.434512,-0.094335,0.058254,-0.172321,-0.004136,0.805680,-0.142349


In [15]:
y

0         1
1         1
2         1
3         1
4         1
         ..
13068    24
13069    24
13070    24
13071    24
13072    24
Name: activity_id, Length: 13073, dtype: int64

In [16]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13073 entries, 0 to 13072
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hand_tmp_mean     13073 non-null  float64
 1   hand_tmp_std      13073 non-null  float64
 2   hand_acc_x_mean   13073 non-null  float64
 3   hand_acc_x_std    13073 non-null  float64
 4   hand_acc_y_mean   13073 non-null  float64
 5   hand_acc_y_std    13073 non-null  float64
 6   hand_acc_z_mean   13073 non-null  float64
 7   hand_acc_z_std    13073 non-null  float64
 8   hand_gyr_x_mean   13073 non-null  float64
 9   hand_gyr_x_std    13073 non-null  float64
 10  hand_gyr_y_mean   13073 non-null  float64
 11  hand_gyr_y_std    13073 non-null  float64
 12  hand_gyr_z_mean   13073 non-null  float64
 13  hand_gyr_z_std    13073 non-null  float64
 14  chest_tmp_mean    13073 non-null  float64
 15  chest_tmp_std     13073 non-null  float64
 16  chest_acc_x_mean  13073 non-null  float6

In [20]:
X = X[[
 'hand_tmp_mean',
 'hand_tmp_std',
 'hand_acc_x_mean',
 'hand_acc_x_std',
 'hand_acc_y_mean',
 'hand_acc_y_std',
 'hand_acc_z_mean',
 'hand_acc_z_std',
 'hand_gyr_x_mean',
 'hand_gyr_x_std',
 'hand_gyr_y_mean',
 'hand_gyr_y_std',
 'hand_gyr_z_mean',
 'hand_gyr_z_std',
 'hand_acc_xy_cor',
 'hand_acc_yz_cor',
 'hand_acc_xz_cor',
 'hand_gyr_xy_cor',
 'hand_gyr_yz_cor',
 'hand_gyr_xz_cor',
 'chest_tmp_mean',
 'chest_tmp_std',
 'chest_acc_x_mean',
 'chest_acc_x_std',
 'chest_acc_y_mean',
 'chest_acc_y_std',
 'chest_acc_z_mean',
 'chest_acc_z_std',
 'chest_gyr_x_mean',
 'chest_gyr_x_std',
 'chest_gyr_y_mean',
 'chest_gyr_y_std',
 'chest_gyr_z_mean',
 'chest_gyr_z_std',
 'chest_acc_xy_cor',
 'chest_acc_yz_cor',
 'chest_acc_xz_cor',
 'chest_gyr_xy_cor',
 'chest_gyr_yz_cor',
 'chest_gyr_xz_cor',
 'ankle_tmp_mean',
 'ankle_tmp_std',
 'ankle_acc_x_mean',
 'ankle_acc_x_std',
 'ankle_acc_y_mean',
 'ankle_acc_y_std',
 'ankle_acc_z_mean',
 'ankle_acc_z_std',
 'ankle_gyr_x_mean',
 'ankle_gyr_x_std',
 'ankle_gyr_y_mean',
 'ankle_gyr_y_std',
 'ankle_gyr_z_mean',
 'ankle_gyr_z_std',
 'ankle_acc_xy_cor',
 'ankle_acc_yz_cor',
 'ankle_acc_xz_cor',
 'ankle_gyr_xy_cor',
 'ankle_gyr_yz_cor',
 'ankle_gyr_xz_cor',]]

In [21]:
X

,hand_tmp_mean,hand_tmp_std,hand_acc_x_mean,hand_acc_x_std,hand_acc_y_mean,hand_acc_y_std,hand_acc_z_mean,hand_acc_z_std,hand_gyr_x_mean,hand_gyr_x_std,...,ankle_gyr_y_mean,ankle_gyr_y_std,ankle_gyr_z_mean,ankle_gyr_z_std,ankle_acc_xy_cor,ankle_acc_yz_cor,ankle_acc_xz_cor,ankle_gyr_xy_cor,ankle_gyr_yz_cor,ankle_gyr_xz_cor
0,30.523682,0.030320,-1.879247,3.534154,4.788385,2.567330,6.990495,1.965119,0.107901,0.466133,...,0.053937,0.474670,0.407518,0.913272,0.264226,-0.113223,0.314868,0.325803,-0.294619,-0.170427
1,30.535889,0.030904,-0.185149,1.377860,5.695728,0.972648,7.785865,0.751240,-0.009273,0.381659,...,0.014152,0.202536,0.046104,0.258117,-0.427727,-0.389651,0.486994,0.194234,-0.368513,-0.292237
2,30.548096,0.026321,0.319354,0.447698,5.626324,0.819042,7.950312,0.582057,-0.037028,0.274945,...,-0.013533,0.102034,0.037852,0.237697,-0.562640,-0.522007,0.588619,0.214233,-0.311803,-0.179082
3,30.555542,0.019659,0.322273,0.326337,5.483047,0.568834,8.098098,0.372352,0.029006,0.152585,...,-0.012446,0.088246,0.053858,0.207330,-0.328764,-0.544402,0.499454,0.187620,-0.438468,-0.192798
4,30.558350,0.015562,0.316246,0.102791,5.702229,0.209482,7.966321,0.143267,0.009624,0.092451,...,-0.009388,0.079133,0.039167,0.198324,0.164706,-0.425811,0.105384,0.008690,-0.389506,-0.293527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13068,25.125000,0.000000,1.388376,4.245269,9.146590,12.844180,-2.399859,5.126979,0.273583,1.697603,...,-0.001855,1.123881,0.076668,2.219342,0.365532,-0.651263,-0.701407,0.239495,0.826156,0.110417
13069,25.125000,0.000000,1.423980,4.319272,9.157108,13.125012,-2.583755,5.019451,0.290182,1.715815,...,-0.033220,1.093644,0.027781,2.123367,0.304992,-0.578913,-0.686521,0.152746,0.813039,0.030027
13070,25.125000,0.000000,1.214250,4.700313,9.182128,14.451389,-2.597204,5.610332,0.244093,1.770493,...,-0.007930,1.032964,0.020923,2.060087,0.230855,-0.572560,-0.640646,0.087238,0.794210,-0.039155
13071,25.125000,0.000000,-0.232018,5.082784,8.528821,13.341294,-1.575550,5.546635,0.113382,1.799821,...,0.068351,0.937876,0.091980,1.941319,0.137410,-0.523310,-0.578442,-0.004136,0.805680,-0.142349


In [22]:
Y =  to_categorical(y)
Y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [23]:
y.map(activity_id )

0               lying
1               lying
2               lying
3               lying
4               lying
             ...     
13068    rope jumping
13069    rope jumping
13070    rope jumping
13071    rope jumping
13072    rope jumping
Name: activity_id, Length: 13073, dtype: object

In [26]:
#X = X.drop(['hr_mean','hr_mean_normal','hr_std','hr_std_normal'], axis = 1)


In [24]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13073 entries, 0 to 13072
Data columns (total 60 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   hand_tmp_mean     13073 non-null  float64
 1   hand_tmp_std      13073 non-null  float64
 2   hand_acc_x_mean   13073 non-null  float64
 3   hand_acc_x_std    13073 non-null  float64
 4   hand_acc_y_mean   13073 non-null  float64
 5   hand_acc_y_std    13073 non-null  float64
 6   hand_acc_z_mean   13073 non-null  float64
 7   hand_acc_z_std    13073 non-null  float64
 8   hand_gyr_x_mean   13073 non-null  float64
 9   hand_gyr_x_std    13073 non-null  float64
 10  hand_gyr_y_mean   13073 non-null  float64
 11  hand_gyr_y_std    13073 non-null  float64
 12  hand_gyr_z_mean   13073 non-null  float64
 13  hand_gyr_z_std    13073 non-null  float64
 14  hand_acc_xy_cor   13073 non-null  float64
 15  hand_acc_yz_cor   13073 non-null  float64
 16  hand_acc_xz_cor   13073 non-null  float6

In [25]:
X = np.array(X)
X

array([[ 3.05236816e+01,  3.03196708e-02, -1.87924650e+00, ...,
         3.25802824e-01, -2.94618717e-01, -1.70427138e-01],
       [ 3.05358887e+01,  3.09038060e-02, -1.85148573e-01, ...,
         1.94233980e-01, -3.68513142e-01, -2.92237175e-01],
       [ 3.05480957e+01,  2.63208052e-02,  3.19354051e-01, ...,
         2.14233053e-01, -3.11803249e-01, -1.79081855e-01],
       ...,
       [ 2.51250000e+01,  0.00000000e+00,  1.21425000e+00, ...,
         8.72375717e-02,  7.94209858e-01, -3.91545895e-02],
       [ 2.51250000e+01,  0.00000000e+00, -2.32018184e-01, ...,
        -4.13589267e-03,  8.05680368e-01, -1.42349488e-01],
       [ 2.51250000e+01,  0.00000000e+00, -2.15614465e+00, ...,
        -1.82046691e-02,  7.59174799e-01, -1.53801581e-01]])

In [26]:
X.shape

(13073, 60)

In [27]:
X = X.reshape(-1,20,3)

In [28]:
X.shape

(13073, 20, 3)

In [29]:
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size = 0.2, random_state = 0, stratify = y)

In [43]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train , test_size = 0.2, random_state = 0, stratify = y_train)

In [39]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8366, 20, 3), (2615, 20, 3), (8366, 25), (2615, 25))

In [40]:
# 2D CNN Model
verbose, epochs, batch_size = 0, 10, 16
model = Sequential()
model.add(Conv1D(16,  2, activation = 'relu', input_shape = X_train[0].shape))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(32, 2, activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(y_train.shape[1], activation='softmax'))


In [44]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')
 # fit network
model.fit(X_train, y_train, epochs=epochs, validation_data=(X_val, y_val), batch_size=batch_size)

Epoch 1/10
419/419 [==============================] - 3s 4ms/step - loss: 0.1564 - accuracy: 0.9473 - val_loss: 0.0521 - val_accuracy: 0.9809
Epoch 2/10
419/419 [==============================] - 1s 4ms/step - loss: 0.1455 - accuracy: 0.9498 - val_loss: 0.0471 - val_accuracy: 0.9809
Epoch 3/10
419/419 [==============================] - 2s 4ms/step - loss: 0.1485 - accuracy: 0.9477 - val_loss: 0.0488 - val_accuracy: 0.9827
Epoch 4/10
419/419 [==============================] - 1s 3ms/step - loss: 0.1296 - accuracy: 0.9547 - val_loss: 0.0478 - val_accuracy: 0.9809
Epoch 5/10
419/419 [==============================] - 1s 3ms/step - loss: 0.1222 - accuracy: 0.9567 - val_loss: 0.0546 - val_accuracy: 0.9821
Epoch 6/10
419/419 [==============================] - 1s 3ms/step - loss: 0.1152 - accuracy: 0.9577 - val_loss: 0.0432 - val_accuracy: 0.9833
Epoch 7/10
419/419 [==============================] - 1s 3ms/step - loss: 0.1134 - accuracy: 0.9592 - val_loss: 0.0474 - val_accuracy: 0.9839
Epoch 

In [32]:
loss , accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
#print(model.summary()) 
score=accuracy*100
print('Accuracy = {}'.format(score))

Accuracy = 99.1204559803009
